In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN9004_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../src')

In [3]:
import models.CNN_model_finetune as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../data/opp4.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)
    
train_data = data['train_data']
test_data = data['test_data']
validation_data = data['validation_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

#index_of_changes = data['index_of_changes_test']
 

In [5]:
train_gt.shape, train_data.shape

((1, 700165, 1), (1, 700165, 77))

## Model

In [6]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 1

convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
''' s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()
'''

s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[0,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)
pos_weight = pos_weight[0]

In [7]:
print pos_weight

2.17746


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [10]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [11]:
model.build_conv_net(pos_weight_values = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [13]:
#load the model
saver = tf.train.Saver(var_list = [
    model.weights['wc0'],model.weights['wc1'],model.weights['wc2'],
    model.bias['bc0'],model.bias['bc1'],model.bias['bc2']
])
saver.restore(sess, '../model/CNN9001_opp/CNN9001_opp-4000')

INFO:tensorflow:Restoring parameters from ../model/CNN9001_opp/CNN9001_opp-4000


In [14]:
trainer.train(max_iter = 20000,
                train_eval_step = 1, 
                validation_eval_step = 1,
                display_step = 1)

Train Batch Evaluation
AUC = 0.596 vs. 0.312
F1 = 0.600 vs. 0.478
Precision = 0.643 vs. 0.314
Recall = 0.562 vs. 0.999
Validation
AUC = 0.461 vs. 0.176
F1 = 0.541 vs. 0.315
Precision = 0.502 vs. 0.187
Recall = 0.587 vs. 1.000
Minibatch Loss= 9.982225
Train Batch Evaluation
AUC = 0.595 vs. 0.316
F1 = 0.600 vs. 0.478
Precision = 0.651 vs. 0.314
Recall = 0.556 vs. 1.000
Validation
AUC = 0.466 vs. 0.193
F1 = 0.541 vs. 0.316
Precision = 0.502 vs. 0.189
Recall = 0.587 vs. 0.957
Minibatch Loss= 9.764697
Train Batch Evaluation
AUC = 0.597 vs. 0.313
F1 = 0.600 vs. 0.478
Precision = 0.652 vs. 0.314
Recall = 0.556 vs. 1.000
Validation
AUC = 0.466 vs. 0.203
F1 = 0.542 vs. 0.315
Precision = 0.506 vs. 0.199
Recall = 0.584 vs. 0.764
Minibatch Loss= 9.554661
Train Batch Evaluation
AUC = 0.583 vs. 0.306
F1 = 0.601 vs. 0.478
Precision = 0.625 vs. 0.314
Recall = 0.578 vs. 1.000
Validation
AUC = 0.463 vs. 0.184
F1 = 0.545 vs. 0.315
Precision = 0.509 vs. 0.187
Recall = 0.587 vs. 0.997
Minibatch Loss= 9.351

KeyboardInterrupt: 

In [15]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/CNN9004_opp/CNN9004_opp-4583'

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../models/model" + exp_name + "/model")

In [16]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

AUC = 0.576 vs. 0.172
F1 = 0.640 vs. 0.315
Precision = 0.557 vs. 0.187
Recall = 0.752 vs. 0.989


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})